<h1>Model - Number of Trips<h1>

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Normalization

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean,count
import numpy as np

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 21:47:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 21:47:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf = spark.read.parquet('../data/curated/combined_data')

In [5]:
df = sdf.groupby('Date', 'Hour', 'PU_Location_ID').agg(avg("Temperature_C").alias("Temperature_C"), \
                                                           avg("Humidity_%").alias("Humidity_%"), \
                                                           avg("Speed_kmh").alias("Speed_kmh"), \
                                                           avg("Precip_Rate_mm").alias("Precip_rate_mm"), \
                                                           avg("Driver_pay").alias("Avg_driver_pay"), \
                                                           avg("Day_of_week").alias("Day_of_week"), \
                                                           count('Temperature_C').alias("Num_trips")).toPandas()

In [6]:
df = df.sort_values(['Date', 'PU_Location_ID', 'Hour'], axis = 0)

In [7]:
df = pd.get_dummies(df, columns=['Hour', 'PU_Location_ID', 'Day_of_week'])

In [8]:
TARGET_COLS = ['Num_trips']

train, test = train_test_split(df, train_size=0.8, shuffle = False)

X_train, y_train = train.drop(TARGET_COLS, axis=1).drop(['Avg_driver_pay', 'Date'], axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1).drop(['Avg_driver_pay', 'Date'], axis=1), test[TARGET_COLS]

In [9]:
norm_layer = Normalization()
norm_layer.adapt(X_train)

Metal device set to: Apple M1


2022-08-16 21:48:07.730975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-16 21:48:07.731157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-16 21:48:15.629715: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-16 21:48:15.748300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-16 21:48:15.769486: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [10]:
model = keras.Sequential(
    [   
        norm_layer,
        Dense(10, activation='relu'),
        Dense(1, activation='relu')
    ]
)

In [11]:
model.compile(
    optimizer='adam',
    loss='MSE'
)

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    validation_split=0.25,
    epochs=10
)

Epoch 1/10
   15/81606 [..............................] - ETA: 4:51 - loss: 23084.3906   

2022-08-16 21:51:19.336661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - ETA: 0s - loss: 3052.3044

2022-08-16 21:56:17.482603: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - 358s 4ms/step - loss: 3052.3044 - val_loss: 2476.2039
Epoch 2/10
81606/81606 [==============================] - 361s 4ms/step - loss: 5234.0190 - val_loss: 3457.3210
Epoch 3/10
81606/81606 [==============================] - 359s 4ms/step - loss: 4897.5342 - val_loss: 3289.9507
Epoch 4/10
25689/81606 [========>.....................] - ETA: 3:28 - loss: 5383.8032

In [ ]:
comparison = y_test.iloc[:10].copy()
comparison.loc[:, 'prediction_num_trips'] = model.predict(X_test.head(10))
comparison

In [ ]:
predictions = model.predict(X_test)
errors = np.array(predictions - y_test)
squared_errors = errors**2
mean_squared_error = squared_errors.mean()

print(f'MSE: {mean_squared_error}')

In [ ]:
tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

<h3>Save predictions for further analysis<h3>

In [ ]:
import os
if not os.path.exists('../data/curated/model_data'):
    os.makedirs('../data/curated/model_data')

In [ ]:
pd.DataFrame(predictions).to_csv('../data/curated/model_data/num_trips_pred.csv')

In [ ]:
test.to_csv('../data/curated/model_data/test_data.csv')